In [1]:
# require to wrap all our import libraries within the spotify function 
from airflow.decorators import dag, task
from airflow import DAG
from datetime import datetime
from datetime import timedelta
from airflow.operators.python import PythonOperator, BranchPythonOperator
from airflow.operators.bash import BashOperator
from airflow.utils.dates import days_ago 
import pathlib
import pandas as pd 
import sqlalchemy 
import configparser as ConfigParser
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [66]:
def check_if_valid_data(df:pd.DataFrame):
       # check if dataframe is empty, it means there were no songs listened to
       if df.empty:
           print('No songs downloaded. Finishing execution')
           return False 
       # Primary key check 
       if pd.Series(df['played_at_list']).is_unique:       # this check helps us understand that there are no duplicate rows in our database
           pass 
       else: 
           raise Exception("Primary Key check if violated")  # here pipeline fail, maybe can send email to me
       yesterday = datetime.now() - timedelta(days = 1)
       # just check yesterday's date at 0 hour, 0 minute, 0 second and 0 microseconds
       yesterday = yesterday.replace(hour = 0,minute = 0,second =0,microsecond =0 )
       # timestamps = df['timestamps'].tolist()
       df = df[df['timestamps']== yesterday.strftime("%Y-%m-%d")]
       songs_json= df.to_json()
       # for timestamp in timestamps:
       #     # strptime --> converts string to time
       #      # if we catch records that are not yesterday, we want the pipeline to raise exception 
       #     if datetime.strptime(timestamp,"%Y-%m-%d")!= yesterday:
       #         raise Exception("At least one of the songs does not come within last 24 hours")
       return songs_json

In [67]:
def update_database(DB_LOCATION,json_data):
        # start database
        # if value:

        songs_table = pd.read_json(json_data)

        engine = sqlalchemy.create_engine(DB_LOCATION)
        # conn = sqlite3.connect('james_played_tracks.sqlite')
        # cursor = conn.cursor()            
        sql_query = """
        CREATE TABLE IF NOT EXISTS james_played_tracks(
            played_at_list VARCHAR(200), 
            timestamps VARCHAR(200),
            artist_name VARCHAR(200), 
            song_names VARCHAR(200),
            CONSTRAINT primary_key_constraint PRIMARY KEY (played_at_list)
        )
        """
        engine.execute(sql_query)
        print("Opened database successfully")  
        # update database 
        try: 
            songs_table.to_sql(name = 'james_played_tracks',con = engine, if_exists= 'append',index = False)
            print("New songs populated in database!")
        except: 
            print("Data already exists in database")
        # conn.close()
        print("database closed successfully")

## Spotipy documentation: 

https://spotipy.readthedocs.io/en/master/#

In [3]:
redirect_uri = 'https://localhost:8888/callback/'
scope= "user-read-recently-played"

In [4]:
current_directory = !pwd

In [5]:
# CURRENT_PATH_DIR= pathlib.Path(!pwd).absolute()
GRANDPARENT_PATH = pathlib.Path(current_directory[0])
CONF_PATH = GRANDPARENT_PATH.joinpath("configuration/config.ini")
cf_parser= ConfigParser.ConfigParser()
cf_parser.read(CONF_PATH)


['/home/maxong/spotify_airflow/configuration/config.ini']

In [6]:
    # TOKEN = cf_parser.get('spotify_details','token')
WEBSITE = cf_parser.get('spotify_details','website')
CLIENT_ID = cf_parser.get('spotify_details','client_id')
CLIENT_SECRET = cf_parser.get('spotify_details','client_secret')
REDIRECT_URL = cf_parser.get('spotify_details','redirect_url')
SCOPE = cf_parser.get('spotify_details','scope')
DB_LOCATION = cf_parser.get('database','db_location_postgres')

In [7]:
config_dictionary = {
'website':WEBSITE,
'client_id':CLIENT_ID,
'client_secret':CLIENT_SECRET,
'redirect_url':REDIRECT_URL,
'scope':SCOPE,
'db_location':DB_LOCATION
}

## Following this stackoverflow to auto refresh the token 

https://stackoverflow.com/questions/48883731/refresh-token-spotipy#:~:text=So%20it%20will%20be%20refreshed,access%20token%20%2F%20refresh%20token%20previously.


Direct link to spotipy utils to follow https://github.com/spotipy-dev/spotipy/blob/master/spotipy/util.py


In [8]:
def create_spotify_api_details(config_dictionary):
    auth_manager=SpotifyOAuth(scope=config_dictionary['scope'],
                                client_id =config_dictionary['client_id'] ,
                                client_secret = config_dictionary['client_secret'],
                                redirect_uri = config_dictionary['redirect_url'])
    spotify = spotipy.Spotify(auth_manager=auth_manager)
    return auth_manager,spotify

   

In [56]:
def refresh_spotify_api_details(auth_manager, spotify,config_dictionary):
   token_info = auth_manager.cache_handler.get_cached_token()
   if auth_manager.is_token_expired(token_info):
       auth_manager, spotify = create_spotify(config_dictionary)
   return auth_manager, spotify

In [62]:
auth_manager, spotify = refresh_spotify_api_details(auth_manager, spotify,config_dictionary)
data = spotify.current_user_recently_played(after =yesterday_unix_timestamp)

In [64]:
while True:
        auth_manager, spotify = refresh_spotify_api_details(auth_manager, spotify,config_dictionary)
        data = spotify.current_user_recently_played(after =yesterday_unix_timestamp)

        try:
            # if python not equals to zero 
            if data['items'] != []:
                artist_name = []
                song_names = []
                played_at_list = []
                timestamps = []         
                for i in data['items']:
                    if i['played_at'][:10]!= today.strftime("%Y-%m-%d"):
                        song_names.append(i['track']['name'])
                        played_at_list.append(i['played_at'])
                        timestamps.append(i['played_at'][:10])
                        artist_name.append(i['track']['artists'][0]['name'])            
                # data is in dataframe format now
                songs_table = pd.DataFrame([played_at_list,timestamps,artist_name,song_names]).T
                songs_table.columns = ['played_at_list','timestamps','artist_name','song_names']            

                # update database
                # update_database(DB_LOCATION,songs_table)
                print(songs_table)

                # once you updated the database, break out of the loop
                break
            # if there are no data that day, don't even open up the database, skip that day    
            else:
                print("No songs played yesterday")
                # if there are no songs, break out 
                break 
        except:
            print("Error with database or spotify data returned")

              played_at_list  timestamps       artist_name  \
0   2022-11-04T12:42:14.577Z  2022-11-04     Taron Egerton   
1   2022-11-04T12:41:51.444Z  2022-11-04     Taron Egerton   
2   2022-11-04T12:38:27.901Z  2022-11-04     Taron Egerton   
3   2022-11-04T12:10:36.062Z  2022-11-04     Taron Egerton   
4   2022-11-04T12:03:04.392Z  2022-11-04     Taron Egerton   
5   2022-11-04T11:59:41.385Z  2022-11-04     Taron Egerton   
6   2022-11-04T11:56:16.822Z  2022-11-04     Taron Egerton   
7   2022-11-04T11:52:53.547Z  2022-11-04     Taron Egerton   
8   2022-11-04T11:49:30.719Z  2022-11-04     Taron Egerton   
9   2022-11-04T11:46:06.196Z  2022-11-04     Taron Egerton   
10  2022-11-04T07:54:15.536Z  2022-11-04     Taron Egerton   
11  2022-11-04T07:52:42.414Z  2022-11-04     Taron Egerton   
12  2022-11-04T07:49:18.444Z  2022-11-04     Taron Egerton   
13  2022-11-04T07:45:54.529Z  2022-11-04     Taron Egerton   
14  2022-11-04T07:42:31.319Z  2022-11-04     Taron Egerton   
15  2022

In [68]:
songs_json= check_if_valid_data(songs_table)

In [71]:
songs_table = pd.read_json(songs_json)


In [72]:
songs_table

,played_at_list,timestamps,artist_name,song_names
0,2022-11-04T12:42:14.577Z,2022-11-04,Taron Egerton,There's Nothing Holdin' Me Back (with Tori Kelly)
1,2022-11-04T12:41:51.444Z,2022-11-04,Taron Egerton,There's Nothing Holdin' Me Back (with Tori Kelly)
2,2022-11-04T12:38:27.901Z,2022-11-04,Taron Egerton,There's Nothing Holdin' Me Back (with Tori Kelly)
3,2022-11-04T12:10:36.062Z,2022-11-04,Taron Egerton,There's Nothing Holdin' Me Back (with Tori Kelly)
4,2022-11-04T12:03:04.392Z,2022-11-04,Taron Egerton,There's Nothing Holdin' Me Back (with Tori Kelly)
5,2022-11-04T11:59:41.385Z,2022-11-04,Taron Egerton,There's Nothing Holdin' Me Back (with Tori Kelly)
6,2022-11-04T11:56:16.822Z,2022-11-04,Taron Egerton,There's Nothing Holdin' Me Back (with Tori Kelly)
7,2022-11-04T11:52:53.547Z,2022-11-04,Taron Egerton,There's Nothing Holdin' Me Back (with Tori Kelly)
8,2022-11-04T11:49:30.719Z,2022-11-04,Taron Egerton,There's Nothing Holdin' Me Back (with Tori Kelly)
9,2022-11-04T11:46:06.196Z,2022-11-04,Taron Egerton,There's Nothing Holdin' Me Back (with Tori Kelly)


In [70]:
engine = sqlalchemy.create_engine(DB_LOCATION)
# conn = sqlite3.connect('james_played_tracks.sqlite')
# cursor = conn.cursor()            
sql_query = """
CREATE TABLE IF NOT EXISTS james_played_tracks(
    played_at_list VARCHAR(200), 
    timestamps VARCHAR(200),
    artist_name VARCHAR(200), 
    song_names VARCHAR(200),
    CONSTRAINT primary_key_constraint PRIMARY KEY (played_at_list)
)
"""
engine.execute(sql_query)
print("Opened database successfully")  

Opened database successfully
